In [1]:
import os
import config_parser

NameError: name 'config' is not defined

In [3]:
import yaml
import pandas as pd

In [4]:
with open("../../wbuild.yaml", 'r') as stream:
    config = yaml.safe_load(stream)
print(config)

{'webDir': '/s/public_webshare/project/genetic_diagnosis', 'projectTitle': 'Genetic diagnosis of Mendelian disorders', 'htmlOutputPath': 'Output/html', 'rawdir_proteome': '/s/project/mitoMultiOmics/raw_data/proteome', 'PROC_DATA': '/s/project/genetic_diagnosis/processed_data', 'PROC_RESULTS': '/s/project/genetic_diagnosis/processed_results', 'RAW_DATA': 'Data/helmholtz', 'RAWDATADIR': 'Data/helmholtz', 'VEPDATADIR': '/s/project/metaIBD/vepAnnoObj/sampleObj', 'SANITYCHECKDIR': '/s/project/mitoMultiOmics/sanitychecks', 'SAMPLE_ANNOTATION': '/s/project/mitoMultiOmics/raw_data/sample_info/SAMPLE_ANNOTATION_PROKISCH.tsv', 'ANNOTATIONS': ['v29_overlap'], 'VEP_VERSION': 94, 'INTER_FEATURE': [0], 'fpkmCutoff': 1}


In [ ]:
def outrider_files(config):

    # sample-file mapping
    # file 
    sa_file = config["SAMPLE_ANNOTATION"]
    
    anno = pd.read_csv(sa_file, sep='\t')
    
    # subset and clean
    anno_outrider = anno[(anno.LAB == "PROKISCH") & pd.notnull(anno.RNA_ID) & pd.notnull(anno.OUTRIDER_GROUP)]
    anno_outrider = anno_outrider[["RNA_ID", "OUTRIDER_GROUP"]].drop_duplicates().copy()

    # create filenames and ignore missing files
    anno_outrider['file'] = [config["RAW_DATA"] + "/" + x + "/RNAout/" for x in anno_outrider["RNA_ID"]]
    anno_outrider['file_exists'] = [os.path.exists(x) for x in anno_outrider["file"]]
    anno_outrider = anno_outrider[anno_outrider['file_exists']]

    # subset by OUTRIDER_GROUP
    outrider_groups = []
    for s in set(anno_outrider.OUTRIDER_GROUP):
        outrider_groups.extend(s.split(','))
    outrider_ids = {og : anno_outrider.loc[anno_outrider.OUTRIDER_GROUP.str.contains('(^|,)' + og + '(,|$)'), 'RNA_ID'].tolist() for og in set(outrider_groups)}
    return outrider_ids, {og: _list for og, _list in outrider_ids.items() if len(_list) > 40}

In [ ]:
if sa_file is None:
        sa_file = config["SAMPLE_ANNOTATION"]
    
    anno = pd.read_csv(sa_file, sep='\t')

In [2]:
def get_files_by_group(config, group):
    return expand(config["PROC_RESULTS"] + "/{{annotation}}/counts/{sampleID}.Rds", sampleID=config["outrider"][group])

In [ ]:
def all_vcf(sa_file = config["SAMPLE_ANNOTATION"]):

    anno = pd.read_csv(sa_file, sep='\t')
    
    # subset and clean
    anno_vcf = anno[(anno.LAB == "PROKISCH") & pd.notnull(anno.EXOME_ID)]
    anno_vcf = anno_vcf[["EXOME_ID"]].copy()
    anno_vcf['file'] = [config["RAW_DATA"] + "/" + x + "/exomicout/" for x in anno_vcf["EXOME_ID"]]
    anno_vcf['vcf_exists'] = [os.path.exists(x) for x in anno_vcf["file"]]
    anno_vcf = anno_vcf[anno_vcf['vcf_exists']]
    
    return anno_vcf["EXOME_ID"].tolist()

In [ ]:
def mae_files(sa_file = config["SAMPLE_ANNOTATION"]):
    
    anno = pd.read_csv(sa_file, sep='\t')
    
    # subset and clean
    anno_mae = anno[anno["LAB"] == "PROKISCH"]
    anno_mae = anno_mae[pd.notnull(anno_mae.EXOME_ID)]
    anno_mae = anno_mae[pd.notnull(anno_mae.RNA_ID)]
    anno_mae = anno_mae[["EXOME_ID", "RNA_ID"]].copy()

    # create file names
    # anno_mae['rna_file'] = [config["RAW_DATA"] + "/" + x + "/RNAout/paired-endout/stdFilenames/" + x + ".bam" for x in anno_mae["RNA_ID"]]
    # anno_mae['vcf_file'] = [config["RAW_DATA"] + "/" + x + "/exomicout/paired-endout/stdFilenames/" + x + ".vcf.gz" for x in anno_mae["EXOME_ID"]]
    
    anno_mae['rna_file'] = [config["RAW_DATA"] + "/" + x + "/RNAout" for x in anno_mae["RNA_ID"]]
    anno_mae['vcf_file'] = [config["RAW_DATA"] + "/" + x + "/exomicout" for x in anno_mae["EXOME_ID"]]

    # check for missing files
    anno_mae['vcf_exists'] = [os.path.exists(x) for x in anno_mae["vcf_file"]]
    anno_mae['rna_exists'] = [os.path.exists(x) for x in anno_mae["rna_file"]]
    anno_mae = anno_mae[anno_mae['vcf_exists'] & anno_mae['rna_exists']]
    
    vcf = anno_mae["EXOME_ID"] 
    rna = anno_mae["RNA_ID"]
    

In [ ]:
def 